## Customer Service Agent for Clothing Retailer

### Authenticate API key:

In [123]:
import os

from dotenv import load_dotenv

load_dotenv()

try:
    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
    if not GOOGLE_API_KEY:
        raise ValueError("GOOGLE_API_KEY not found in .env file")


    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(f"🔑 Authentication Error: Please check your .env file. Details: {e}")


✅ Gemini API key setup complete.


### Import ADK components

In [124]:
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import load_memory, preload_memory
from google.adk.runners import Runner
from google.adk.tools import google_search, AgentTool, ToolContext
from google.adk.code_executors import BuiltInCodeExecutor


from google.adk.plugins.logging_plugin import LoggingPlugin


print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


#### Define Retry Configurations for LLM calls

In [125]:
retry_config = types.HttpRetryOptions(
    attempts=5,  
    exp_base=7,  
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  
)

## Custom Tools

We will define custom tools to be used by our customer service agent. 1. Purchase History, 2. Customer Returns, 3. Item Recommendations

#### 1. Tool to fetch user's purchase history

#### Purchase history database
First we define a purchase_history_database to be used for testing. This is a list of dictionaries which include the userid, item purchased and the item's UPC.

In [126]:
purchase_history_database = { "abigail454": {
        "red dress": "UI843972429",
        "blue jeans": "UP20928392",
        "white sneakers": "SN938475610"
    },
        "nick123": {
        "black leather jacket": "JL564738291",
        "gray hoodie": "HD837465920",
        "running shorts": "RS102938475"
    },
        "sophia789": {
        "floral skirt": "FS192837465",
        "denim jacket": "DJ564738920",
        "ankle boots": "AB837465192"
    }
       }


In [127]:
# Find Customer's purchase history using customer id as input

def get_purchase_history(user_id: str) -> dict:
    """
    Looks up the customer's purchase history

    Args:
        user_id (str): The customer's userid {"userid"}

    Returns:
        dict:
            Success -> {"status": "success", "purchase_history": {"item": "UPC"}}
            Error   -> {"status": "error", "error_message": "Customer history not found"}
    """

    # Lookup
    purchase_history = purchase_history_database.get(user_id)

    if purchase_history is not None:
        return {"status": "success", "purchase_history": purchase_history}
    else:
        return {
            "status": "error",
            "error_message": "No such customer found"
        }


# ✅ Test the tool
print("✅ Customer purchase history tool created")
print(f"💳 Test (abigail454): {get_purchase_history('abigail454')}")
print(f"💳 Test (nick123): {get_purchase_history('nick123')}")
print(f"💳 Test (sophia789): {get_purchase_history('sophia789')}")
print(f"💳 Test (unknown_user): {get_purchase_history('unknown_user')}")



✅ Customer purchase history tool created
💳 Test (abigail454): {'status': 'success', 'purchase_history': {'red dress': 'UI843972429', 'blue jeans': 'UP20928392', 'white sneakers': 'SN938475610'}}
💳 Test (nick123): {'status': 'success', 'purchase_history': {'black leather jacket': 'JL564738291', 'gray hoodie': 'HD837465920', 'running shorts': 'RS102938475'}}
💳 Test (sophia789): {'status': 'success', 'purchase_history': {'floral skirt': 'FS192837465', 'denim jacket': 'DJ564738920', 'ankle boots': 'AB837465192'}}
💳 Test (unknown_user): {'status': 'error', 'error_message': 'No such customer found'}


### 2. Tool that fetches the user's returns

First we define a customer returns database to be used for testing. This is a list of dictionaries which each include customer id, item description and item UPC for each item returned by the customer.

In [128]:
# Customer returns database
customer_returns_database = {
        "abigail454": {
            "blue jeans": "UP20928392"   # Returned item
        },
        "nick123": {
            # No returns yet
        },
        "sophia789": {
            # No returns yet
        }
    }

In [129]:
# Find items returned by the customer

def get_customer_returns(user_id: str) -> dict:
    """
    Looks up the customer's returns

    Args:
        user_id (str): The customer's userid {"userid"}

    Returns:
        dict:
            Success -> {"status": "success", "customer_returns": {"item": "UPC"}}
            Error   -> {"status": "error", "error_message": "Customer not found"}
    """

 # Lookup
    customer_returns = customer_returns_database.get(user_id)

    if customer_returns is not None:
        return {"status": "success", "customer_returns": customer_returns}
    else:
        return {
            "status": "error",
            "error_message": "No such customer found"
        }


# ✅ Test the tool
print("✅ Customer returns lookup tool created")
print(f"💳 Test (abigail454): {get_customer_returns('abigail454')}")
print(f"💳 Test (nick123): {get_customer_returns('nick123')}")
print(f"💳 Test (sophia789): {get_customer_returns('sophia789')}")
print(f"💳 Test (unknown_user): {get_customer_returns('unknown_user')}")


✅ Customer returns lookup tool created
💳 Test (abigail454): {'status': 'success', 'customer_returns': {'blue jeans': 'UP20928392'}}
💳 Test (nick123): {'status': 'success', 'customer_returns': {}}
💳 Test (sophia789): {'status': 'success', 'customer_returns': {}}
💳 Test (unknown_user): {'status': 'error', 'error_message': 'No such customer found'}


### 3. Item Reommendation Tool

#### Product Database 
Here we define a product_database to be used for testing. This contains item descriptions and their associated UPC numbers. 

In [130]:

# A sample product database for testing. This includes item description and a UPC number to identify each item, including its size and colour.
product_database = {
    "red dress": {
        "black dress": "BD837465920",
        "summer dress": "SD564738291",
        "cocktail dress": "CD102938475"
    },
    "blue jeans": {
        "denim jacket": "DJ564738920",
        "skinny jeans": "SJ837465192",
        "cargo pants": "CP192837465"
    },
    "white sneakers": {
        "running shoes": "RS938475610",
        "canvas sneakers": "CS564738920",
        "slip-ons": "SO837465192"
    },
    "black leather jacket": {
        "bomber jacket": "BJ564738291",
        "motorcycle jacket": "MJ837465920",
        "trench coat": "TC102938475"
    },
    "gray hoodie": {
        "zip hoodie": "ZH564738920",
        "pullover hoodie": "PH837465192",
        "sweatshirt": "SW192837465"
    },
    "running shorts": {
        "track pants": "TP564738291",
        "gym shorts": "GS837465920",
        "compression leggings": "CL102938475"
    },
    "floral skirt": {
        "pleated skirt": "PS564738920",
        "mini skirt": "MS837465192",
        "maxi skirt": "MX192837465"
    },
    "denim jacket": {
        "leather jacket": "LJ564738291",
        "windbreaker": "WB837465920",
        "parka": "PK102938475"
    },
    "ankle boots": {
        "chelsea boots": "CB564738920",
        "combat boots": "CO837465192",
        "loafers": "LF192837465"
    }
}


#### Product Style Database 
This database lists items in categories. This is a list of lists. Each list is a group of items that have at least two similar attributes, e.g. style, colour, item type.

In [131]:
product_style_database = [["red dress", "black dress", "summer dress", "cocktail dress"],
                          ["black leather jacket", "bomber jacket", "motorcycle jacket"],
                        ]


### Item Recommendation Tool Creation

The following tool creates item suggestions based on the user's past purchases, with any returned items removed. It compares items by checking the product_style_database. If an item comes up twice in two different groups, we will only add it once to the list of suggested items.

In [132]:
def get_item_recommendation(user_id: str) -> dict:
    """
    Suggests clothing items (with UPCs) similar to those previously purchased,
    excluding items the customer has returned. Uses product_style_database groups.
    """
    purchase_history = purchase_history_database.get(user_id)
    returns = customer_returns_database.get(user_id)

    if purchase_history is None:
        return {"status": "error", "error_message": "No such customer found"}

    suggested_items = {}

    for item in purchase_history.keys():
        # Skip items the customer has returned
        if returns and item in returns:
            continue

        # Find the style group containing this item
        for group in product_style_database:
            if item in group:
                # Suggest other items in the same group
                for similar_item in group:
                    if similar_item != item:  # don't suggest the exact same item
                        if similar_item not in suggested_items:
                            suggested_items[similar_item] = product_database.get(similar_item)


    suggested_items = list(suggested_items.items())

    # Limit to at most 6 items
    suggested_items_list = suggested_items_list[:6]

    return {"status": "success", "suggested_items": suggested_items}


# ✅ Test the tool
print("✅ Product recommendation tool created")
print(f" Test (abigail454): {suggest_similar_items('abigail454')}")
print(f" Test (nick123): {suggest_similar_items('nick123')}")
print(f" Test (sophia789): {suggest_similar_items('sophia789')}")
print(f" Test (unknown_user): {suggest_similar_items('unknown_user')}")



✅ Product recommendation tool created
 Test (abigail454): {'status': 'success', 'suggested_items': [('black dress', None), ('summer dress', None), ('cocktail dress', None)]}
 Test (nick123): {'status': 'success', 'suggested_items': [('bomber jacket', None), ('motorcycle jacket', None)]}
 Test (sophia789): {'status': 'success', 'suggested_items': []}
 Test (unknown_user): {'status': 'error', 'error_message': 'No such customer found'}


#### Helper Function

Here we add a helper function that prints the generated Python code and results from the code execution tool:

In [133]:
def show_python_code_and_result(response):
    for i in range(len(response)):
        # Check if the response contains a valid function call result from the code executor
        if (
            (response[i].content.parts)
            and (response[i].content.parts[0])
            and (response[i].content.parts[0].function_response)
            and (response[i].content.parts[0].function_response.response)
        ):
            response_code = response[i].content.parts[0].function_response.response
            if "result" in response_code and response_code["result"] != "```":
                if "tool_code" in response_code["result"]:
                    print(
                        "Generated Python Code >> ",
                        response_code["result"].replace("tool_code", ""),
                    )
                else:
                    print("Generated Python Response >> ", response_code["result"])


print("✅ Helper functions defined.")

✅ Helper functions defined.


### Cutomer Service Agent

Here we will define the customer AI agent for shopping recommendations, fetching user purchase history and returns.

In [134]:
# Here we have added ADK's built-in Memory Service for development and testing

memory_service = (InMemoryMemoryService())  

# Create Session Service
session_service = InMemorySessionService()  # Handles conversations

print("✅ Agent and Runner created with memory support!")

✅ Agent and Runner created with memory support!


Create auto-callback, so that we can save each session to memory

In [135]:
async def auto_save_to_memory(callback_context):
    """Automatically save session to memory after each agent turn."""
    await callback_context._invocation_context.memory_service.add_session_to_memory(
        callback_context._invocation_context.session
    )

print("✅ Callback created.")

✅ Callback created.


In [136]:
# Customer Service agent for fashion retailer with custom function tools
customer_service_agent = LlmAgent(
    name="customer_agent",
    model=Gemini(model="gemini-2.5-flash-lite", 
                 retry_options=retry_config),
    instruction="""You are a smart customer service assistant for a fashion retailer. You respond in a friendly and enthusiastic manner. You are always polite.

    If a customer asks for item recommendations use:
    2. get_purchase_history() to see what they have purchased in the past
    3. get_customer_returns() to check that the items you find have not been returned by the customer 
    3. get_item_recommendation() to find items simlar to what they have liked in the past and get item recommendations

    If any tool returns status "error", explain the issue to the user clearly.
    """,
    tools=[get_item_recommendation, get_purchase_history, get_customer_returns, preload_memory],
    # Save memory after each turn

    after_agent_callback=auto_save_to_memory        
)

print("✅ Customer service agent created with product recommendation tools")
print("🔧 Available tools:")
print("  • get_suggestion - Looks up items the custome might be interested in")
print("  • get_returns - Looks up items the customer has returned")

✅ Customer service agent created with product recommendation tools
🔧 Available tools:
  • get_suggestion - Looks up items the custome might be interested in
  • get_returns - Looks up items the customer has returned


#### Customer Service Agent Runner with Logging Plugin

Here we create the runner with logging. We test the runner with user input "My user id is abigail454 please recommend some items for me to purchase."

In [137]:
from google.genai import types
import asyncio

APP_NAME = "memory_app"

async def main():
    # Configure the runner
    runner = Runner(
        agent=customer_service_agent,
        app_name = APP_NAME,
        plugins=[LoggingPlugin()],   # Observability logging
        session_service=session_service,
        memory_service=memory_service
    )

    # Run a debug prompt
    result = await runner.run_debug(
        "My user id is abigail454 please recommend some items for me to purchase."
    )

print("✅ Runner configured")
print("🔎 Debug result:", result)

result = await main()


✅ Runner configured
🔎 Debug result: None

 ### Created new session: debug_session_id

User > My user id is abigail454 please recommend some items for me to purchase.
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-5697cf06-b178-42a3-aabb-f17ed653207c
[logging_plugin]    Session ID: debug_session_id
[logging_plugin]    User ID: debug_user_id
[logging_plugin]    App Name: memory_app
[logging_plugin]    Root Agent: customer_agent
[logging_plugin]    User Content: text: 'My user id is abigail454 please recommend some items for me to purchase.'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-5697cf06-b178-42a3-aabb-f17ed653207c
[logging_plugin]    Starting Agent: customer_agent
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: customer_agent
[logging_plugin]    Invocation ID: e-5697cf06-b178-42a3-aabb-f17ed653207c
[logging_plugin] 🧠 LLM REQUEST
[logging_plugin]    Model: gemini-2.5-flash-lite
[logging_plugin]    Agent

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: customer_agent
[logging_plugin]    Content: function_call: get_purchase_history | function_call: get_customer_returns
[logging_plugin]    Token Usage - Input: 481, Output: 46
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: 9267f9b7-284f-476d-97c0-fc8d79dd9c3c
[logging_plugin]    Author: customer_agent
[logging_plugin]    Content: function_call: get_purchase_history | function_call: get_customer_returns
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['get_purchase_history', 'get_customer_returns']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: get_purchase_history
[logging_plugin]    Agent: customer_agent
[logging_plugin]    Function Call ID: adk-8956e277-efd0-45cb-95ad-5372562a0071
[logging_plugin]    Arguments: {'user_id': 'abigail454'}
[logging_plugin] 🔧 TOOL COMPLETED
[logging_plugin]    Tool Name: get_purchase_history
[logging_plugin]    Agent: customer_agent
[l

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: customer_agent
[logging_plugin]    Content: text: 'I would be happy to recommend some items for you, abigail454!

Based on your purchase history, I see you have previously bought blue jeans, a red dress, and white sneakers. I also see that the blue je...' | function_call: get_item_recommendation
[logging_plugin]    Token Usage - Input: 631, Output: 100
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: d3ba1c8f-e166-4fb5-bba2-29242ba7715a
[logging_plugin]    Author: customer_agent
[logging_plugin]    Content: text: 'I would be happy to recommend some items for you, abigail454!

Based on your purchase history, I see you have previously bought blue jeans, a red dress, and white sneakers. I also see that the blue je...' | function_call: get_item_recommendation
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['get_item_recommendation']
customer_agent > I would be happy to recommend some ite

UnboundLocalError: cannot access local variable 'suggested_items_list' where it is not associated with a value

## Test the Agent!

To run a prompt, simply change the value of the "prompt" variable below, and run the cell.

In [ ]:
prompt = "My user id is abigail454 please recommend some items for me to purchase."

result = await runner.run_debug(prompt)

print("🔎 Debug result:", result)
